In [0]:
!pip install pulp #installing pulp if it is not present yet

     |████████████████████████████████| 39.2MB 100kB/s 


In [0]:
import numpy as np #importing required libraries
import pandas as pd
from scipy.optimize import linprog
from pulp import *

# PuLP model

Block below contains input data for our problem specified in form of dictionaries to be consumed by PuLP package later. For convinience we've split all variables per primary skill.

In [0]:
# Generated data for PuLP

# Creates a lists of vvariables for Developers, QA and BA. Pattern to name variables is AA(A)BC(C), where AA(A) - 2 or 3 characters code of Delivery Location, B - Indicates Level of the resource primary skill, 
# C(C) - 1 or 2 characters indicating primary skill of resource.
resources_dev = ['USEJD', 'USEMD', 'USESD', 'USWJD', 'USWMD', 'USWSD', 'UKJD', 'UKMD', 'UKSD', 'INJD', 'INMD', 'INSD']
resources_qa = ['USEJQA', 'USEMQA', 'USESQA', 'USWJQA', 'USWMQA', 'USWSQA', 'UKJQA', 'UKMQA', 'UKSQA', 'INJQA', 'INMQA', 'INSQA']
resources_ba = ['USEJBA', 'USEMBA', 'USESBA', 'USWJBA', 'USWMBA', 'USWSBA', 'UKJBA', 'UKMBA', 'UKSBA', 'INJBA', 'INMBA', 'INSBA']

# A dictionaries of the costs of each set of variables
costs_dev = {'USEJD': 500, 
            'USEMD': 650, 
            'USESD': 800, 
            'USWJD': 600, 
            'USWMD': 750, 
            'USWSD': 900,
            'UKJD': 700, 
            'UKMD': 850, 
            'UKSD': 1000, 
            'INJD': 100, 
            'INMD': 200, 
            'INSD': 300}

costs_qa = {'USEJQA': 450, 
            'USEMQA': 550, 
            'USESQA': 700, 
            'USWJQA': 550, 
            'USWMQA': 650, 
            'USWSQA': 800,
            'UKJQA': 600, 
            'UKMQA': 750, 
            'UKSQA': 800, 
            'INJQA': 90, 
            'INMQA': 180, 
            'INSQA': 270}

costs_ba = {'USEJBA': 550, 
            'USEMBA': 700, 
            'USESBA': 900, 
            'USWJBA': 550, 
            'USWMBA': 700, 
            'USWSBA': 850,
            'UKJBA': 750, 
            'UKMBA': 950, 
            'UKSBA': 1100, 
            'INJBA': 150, 
            'INMBA': 250, 
            'INSBA': 350}

# A dictionary of the final availability of resources calculated as headcount multiplied by availability
availability_dev = {'USEJD': 3*1, 
                    'USEMD': 2*1, 
                    'USESD': 1*0.5, 
                    'USWJD': 3*1, 
                    'USWMD': 2*0.5, 
                    'USWSD': 1*0.5,
                    'UKJD': 4*0.5, 
                    'UKMD': 3*1, 
                    'UKSD': 2*1, 
                    'INJD': 10*1, 
                    'INMD': 8*1, 
                    'INSD': 6*0.5}

availability_qa = {'USEJQA': 2*1, 
                    'USEMQA': 2*0.5, 
                    'USESQA': 2*1, 
                    'USWJQA': 2*0.5, 
                    'USWMQA': 2*1, 
                    'USWSQA': 2*0.5,
                    'UKJQA': 3*1, 
                    'UKMQA': 3*1, 
                    'UKSQA': 3*0.5, 
                    'INJQA': 8*0.5, 
                    'INMQA': 8*1, 
                    'INSQA': 8*0.5}

availability_ba = {'USEJBA': 4*0.5, 
                    'USEMBA': 1*1, 
                    'USESBA': 1*1, 
                    'USWJBA': 1*1, 
                    'USWMBA': 3*1, 
                    'USWSBA': 2*0.5,
                    'UKJBA': 3*1, 
                    'UKMBA': 4*0.5, 
                    'UKSBA': 2*1, 
                    'INJBA': 7*0.5, 
                    'INMBA': 10*0.5, 
                    'INSBA': 7*0.5}

Below we are defining main model to carry out experiments with regards to Project Sourcing problem. Model defined using PuLP package since it better works with large amount of variables and constraints. All constraints we assume for this problem are implemented here.

In [0]:
def solve_problem(method, constraints_dict): # defining model using PuLP for experiments with our problem
  prob = LpProblem("The Sourcing Problem", LpMaximize) # creeating maximization problem

  resources_vars_dev = LpVariable.dicts("Developer",resources_dev,0, cat="Integer") #Seting up variables for Developers, QAs and BAs skills
  resources_vars_qa = LpVariable.dicts("QA",resources_qa,0, cat="Integer")
  resources_vars_ba = LpVariable.dicts("BA",resources_ba,0, cat="Integer")

  # adding optimization equation
  prob += lpSum([costs_dev[i]*resources_vars_dev[i] for i in resources_dev]) + lpSum([costs_qa[i]*resources_vars_qa[i] for i in resources_qa]) + lpSum([costs_ba[i]*resources_vars_ba[i] for i in resources_ba]), "Total Cost of Ingredients per can"

  # constraint on total cost of sourcing plan
  if constraints_dict['tot_cost'] != None:
    prob += lpSum([costs_dev[i]*resources_vars_dev[i] for i in resources_dev]) + lpSum([costs_qa[i]*resources_vars_qa[i] for i in resources_qa]) + lpSum([costs_ba[i]*resources_vars_ba[i] for i in resources_ba]) - constraints_dict['tot_cost'] <= 0, "Total cost per day limit"
  
  # constraint on availability  of particular resources. If enabled, availability is check against respective dictionary
  if constraints_dict['availability_dev'] == True:
    for var in resources_dev:
      prob += resources_vars_dev[var] <= availability_dev[var], "Developer availability "+var
  if constraints_dict['availability_qa'] == True:
    for var in resources_qa:
      prob += resources_vars_qa[var] <= availability_qa[var], "QA availability "+var
  if constraints_dict['availability_ba'] == True:
    for var in resources_ba:
      prob += resources_vars_ba[var] <= availability_ba[var], "BA availability "+var

  # constraint on total headcount of developers
  if constraints_dict['headcount_dev'] != None:
    prob += lpSum([resources_vars_dev[i] for i in resources_dev]) == constraints_dict['headcount_dev'], "Developer headcount"
  # constraint on total headcount of QAs
  if constraints_dict['headcount_qa'] != None:
    prob += lpSum([resources_vars_qa[i] for i in resources_qa]) == constraints_dict['headcount_qa'], "QA headcount"
  # constraint on total headcount of BAs
  if constraints_dict['headcount_ba'] != None:
    prob += lpSum([resources_vars_ba[i] for i in resources_ba]) == constraints_dict['headcount_ba'], "BA headcount"

  # constraint on Junior Level resources
  if constraints_dict['junior_dev'] != None:
    prob += lpSum([resources_vars_dev[i] for i in resources_dev if i[-2] == 'J']) >= constraints_dict['junior_dev'][0], "Junior Developer lower"
    prob += lpSum([resources_vars_dev[i] for i in resources_dev if i[-2] == 'J']) <= constraints_dict['junior_dev'][1], "Junior Developer upper"
  if constraints_dict['junior_qa'] != None:
    prob += lpSum([resources_vars_qa[i] for i in resources_qa if i[-3] == 'J']) >= constraints_dict['junior_qa'][0], "Junior QA lower"
    prob += lpSum([resources_vars_qa[i] for i in resources_qa if i[-3] == 'J']) <= constraints_dict['junior_qa'][1], "Junior QA upper"
  if constraints_dict['junior_ba'] != None:
    prob += lpSum([resources_vars_ba[i] for i in resources_ba if i[-3] == 'J']) >= constraints_dict['junior_ba'][0], "Junior BA lower"
    prob += lpSum([resources_vars_ba[i] for i in resources_ba if i[-3] == 'J']) <= constraints_dict['junior_ba'][1], "Junior BA upper"

  # constraint on Middle Level resources
  if constraints_dict['middle_dev'] != None: 
    prob += lpSum([resources_vars_dev[i] for i in resources_dev if i[-2] == 'M']) >= constraints_dict['middle_dev'][0], "Middle Developer lower"
    prob += lpSum([resources_vars_dev[i] for i in resources_dev if i[-2] == 'M']) <= constraints_dict['middle_dev'][1], "Middle Developer upper"
  if constraints_dict['middle_qa'] != None:
    prob += lpSum([resources_vars_qa[i] for i in resources_qa if i[-3] == 'M']) >= constraints_dict['middle_qa'][0], "Middle QA lower"
    prob += lpSum([resources_vars_qa[i] for i in resources_qa if i[-3] == 'M']) <= constraints_dict['middle_qa'][1], "Middle QA upper"
  if constraints_dict['middle_ba'] != None:
    prob += lpSum([resources_vars_ba[i] for i in resources_ba if i[-3] == 'M']) >= constraints_dict['middle_ba'][0], "Middle BA lower"
    prob += lpSum([resources_vars_ba[i] for i in resources_ba if i[-3] == 'M']) <= constraints_dict['middle_ba'][1], "Middle BA upper"

  # constraint on Senior Level resources
  if constraints_dict['senior_dev'] != None:
    prob += lpSum([resources_vars_dev[i] for i in resources_dev if i[-2] == 'S']) >= constraints_dict['senior_dev'][0], "Senior Developer lower"
    prob += lpSum([resources_vars_dev[i] for i in resources_dev if i[-2] == 'S']) <= constraints_dict['senior_dev'][1], "Senior Developer upper"
  if constraints_dict['senior_qa'] != None:
    prob += lpSum([resources_vars_qa[i] for i in resources_qa if i[-3] == 'S']) >= constraints_dict['senior_qa'][0], "Senior QA lower"
    prob += lpSum([resources_vars_qa[i] for i in resources_qa if i[-3] == 'S']) <= constraints_dict['senior_qa'][1], "Senior QA upper"
  if constraints_dict['senior_ba'] != None:
    prob += lpSum([resources_vars_ba[i] for i in resources_ba if i[-3] == 'S']) >= constraints_dict['senior_ba'][0], "Senior BA lower"
    prob += lpSum([resources_vars_ba[i] for i in resources_ba if i[-3] == 'S']) <= constraints_dict['senior_ba'][1], "Senior BA upper"

  # constraint on speecific Delivery Location for resources
  if constraints_dict['location_dev'] != None:
    prob += lpSum([resources_vars_dev[i] for i in resources_dev if i[:2] not in constraints_dict['location_dev']]) == 0 , "Developers from certain location"
  if constraints_dict['location_qa'] != None:
    prob += lpSum([resources_vars_qa[i] for i in resources_qa if i[:2] not in constraints_dict['location_qa']]) == 0 , "QAs from certain location"
  if constraints_dict['location_ba'] != None:
    prob += lpSum([resources_vars_ba[i] for i in resources_ba if i[:2] not in constraints_dict['location_ba']]) == 0 , "BAs from certain location"


  prob.solve(solver = PULP_CBC_CMD(options=[method])) # solving problem using method from input
  print("Status:", LpStatus[prob.status]) # prinmnting solution status

  for v in prob.variables(): # printing solution found skipping variables with 0 values
    if v.varValue != 0:
      print(v.name, "=", v.varValue)
      
  print("Total Cost is", value(prob.objective)) # printing final total cost of team per day

  return prob

Below block shows how model should be consumed using dictionary with constraints and respective values. By using this dictionary we could not only disable constraints which are no longer a priority for us, but also easily change constraints boundaries to allow easy and rapid experiments with different sets of constraints. For each constraint we add short comment which instructs how to use it.

In [9]:
constraints = {
    'tot_cost': 9000, # total cost constraint. If None - constraint is disabled. If some other value then it is absolute max amount of total cost
    'availability_dev': True, # availability constraint per resource checked against specific dictionary. Value indicates whether it is enabled (True) or disabled (False, None)
    'availability_qa': True,
    'availability_ba': True,
    'headcount_dev': 7, # constraint on developers headcount. Number represents exact amount of required developers. If None - constraint disabled.
    'headcount_qa': 5, # constraint on QAs headcount. Number represents exact amount of required QAs. If None - constraint disabled.
    'headcount_ba': 3, # constraint on BAs headcount. Number represents exact amount of required BAs. If None - constraint disabled.
    'junior_dev': (0, 4), # constraint on amount of Junior level developers. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 4).
    'junior_qa': (0, 3), # constraint on amount of Junior level QAs. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 3).
    'junior_ba': (0, 1), # constraint on amount of Junior level BAs. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 1).
    'middle_dev': (0, 2), # constraint on amount of Middle level developers. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 2).
    'middle_qa': (0, 1), # constraint on amount of Middle level QAs. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 1).
    'middle_ba': (0, 1), # constraint on amount of Middle level BAs. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 1).
    'senior_dev': (0, 1), # constraint on amount of Senior level developers. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 1).
    'senior_qa': (0, 1), # constraint on amount of Senior level QAs. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 1).
    'senior_ba': (0, 1), # constraint on amount of Senior level BAs. Numbers indicates lower and upper bounds of interval. If None - constraint disabled. Valid example (0, 1).
    'location_dev': ['IN', 'UK'], # constraint on specific location. List represents allowed locations for developers. If None - constraint disabled. Valid example ['IN', 'UK'].
    'location_qa': ['UK'], # constraint on specific location. List represents allowed locations for QAs. If None - constraint disabled. Valid example ['UK'].
    'location_ba': ['US'] # constraint on specific location. List represents allowed locations for BAs. If None - constraint disabled. Valid example ['US'].
}

problem = solve_problem(method='dualSimplex', constraints_dict=constraints) # solve defined problem

# print(problem)

Status: Optimal
BA_USEMBA = 1.0
BA_USESBA = 1.0
BA_USWJBA = 1.0
Developer_INJD = 4.0
Developer_UKMD = 2.0
Developer_UKSD = 1.0
QA_UKJQA = 3.0
QA_UKMQA = 1.0
QA_UKSQA = 1.0
Total Cost is 8600.0


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Result of the model shows values of variables which are not equal to 0. Those variables infdicates team decomposition to fulfill given constraints. If required, problem itself could be printed for more detailed examination. 

# SciPy model

Below model for experiments created with SciPy package. It is simplified model which uses only part of dataset (only data about developers). We simplified it because SciPy package is cumbersome when you need to work with big number of variables or constraints. However, since it is works directly with constraint matrix $A$ and RHS vector $b$ it fits better to showcase our idea of priority matrix $W$, which is used to disable constraint which are not a priority any more. As described in report main idea here is left multiplication of $Ax=b$ with matrix $W$ to remove cosntraints without priority.

In [0]:
def solve_problem_with_matricies(W_matrix): #defining problem solver function with scipy

  c = [-500, -650, -800, -600, -750, -900, -700, -850, -1000, -100, -200, -300] # specifying function for maximization
  A = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], # constraint on total number of resource
      [500, 650, 800, 600, 750, 900, 700, 850, 1000, 100, 200, 300], # upper bound for total cost of team per day
      [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # number of developers from US East
      [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0], # number of developers from US West
      [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0], # number of developers from UK
      [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]] # number of developers from India
  b = [10,
      7000,
      4,
      4,
      4,
      4]
  x0_bounds = (0, 3) #lower and upper bound of our variables representing availability of resources.
  x1_bounds = (0, 2)
  x2_bounds = (0, 0.5)
  x3_bounds = (0, 3)
  x4_bounds = (0, 1)
  x5_bounds = (0, 0.5)
  x6_bounds = (0, 2)
  x7_bounds = (0, 3)
  x8_bounds = (0, 2)
  x9_bounds = (0, 10)
  x10_bounds = (0, 8)
  x11_bounds = (0, 3)

  bounds_all = [x0_bounds, x1_bounds, x2_bounds, x3_bounds, x4_bounds, x5_bounds, x6_bounds, x7_bounds, x8_bounds, x9_bounds, x10_bounds, x11_bounds]

  res = linprog(c, A_ub=np.dot(W_matrix, A), b_ub=np.dot(W_matrix, b), bounds=bounds_all, method='simplex')

  return res

Now executing model with all constraints active. It means priority matrix $W=I_n$, where $n$ is number of constraints.

In [0]:
W = [ # ,atrix with all caontraints active
     [1, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 0, 1, 0, 0, 0],
     [0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1]
     ]

solution = solve_problem_with_matricies(W)

print("Solution found for vector x=%s" %solution.x.round(3))

Solution found for vector x=[0.    0.    0.5   2.5   1.    0.5   0.    1.545 2.    0.    0.    1.955]


As result we got numbers of resource to staff which are not represents half-day or full-day load. In such case we will need to perform further interpretation of results and round variables.

Now, let assume that constraints for number of developers from US location are no longer priority for us. So we are removing respective lines from priority matrix $W$.

In [0]:
W = [ # matrix with disabled constraints for Us location
     [1, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1]
     ]

solution = solve_problem_with_matricies(W)

print("Solution found for vector x=%s" %solution.x)

Solution found for vector x=[0.  2.  0.5 0.  1.  0.5 0.  2.  2.  0.  2.  0. ]


As result  after we removed US location constraints, which are not a priority for us anylonger, we got a different team composition setup which fits better to our needs. 